In [1]:
# rnn geneliklemetin karşılaştırma ve sınıflandırmada kullanılır.

# 1. Kütüphane ve veri

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1.1 Veri seti Bilgi

    https://www.kaggle.com/iarunava/imdb-movie-reviews-dataset
    50,000 film değerlendirmesi/yorumu var. Bunlardan 25 bin tanesi 5 puandan düşük almış (olumsuz değerlendirme), 25 bin tanesi 5 puandan büyük almış (olumlu değerlendirme).
    
    Problem:
    Yorumları okuyarak yorumun olumlu olup olmadığını tahmin etmek.

# 2. Veri Ön Hazırlığı

## 2.1. Veri seti parametrelerini ayarlama

In [3]:
# Yorumların uzunlukları aynı değil. Ancak tüm girdilerin aynı uzunlukta olması gerekir.
number_of_words = 20000
max_len = 100

## 2.2. imdb veri setini yükleme

In [4]:
# (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=number_of_words)
# kodunda ValueError: Object arrays cannot be loaded when allow_pickle=False
# hatası oluştu. Bunu önlemek için aşağıdaki kodları çalıştırmalıyız.

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [5]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=number_of_words)

In [6]:
# Veriyi hatasız okuduktan sonra değişiklikleri eski haline getirelim
np.load = np_load_old

## 2.3. Padding all sequences to be same length

In [7]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

In [8]:
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

X_train:  (25000, 100)
y_train:  (25000,)
X_test:  (25000, 100)
y_test:  (25000,)


# 3. Özyinelemeli (Tekrarlayan) Sinir Ağını Örmek 
# (Building an Recurrent Neural Network - RNN)

## 3.1. Model

In [9]:
model = tf.keras.models.Sequential()

## 3.2. Embedding katmanını ekleme

In [10]:
# Embedding layer:

In [11]:
model.add(tf.keras.layers.Embedding(
                                    input_dim=number_of_words, 
                                    output_dim=128,
                                    input_shape=(X_train.shape[1],)                                   
                                   )
         )

## 3.3. LSTM katmanını ekleme

In [12]:
# units: 128
# activation: tanh (hyperbolic tanjant)

In [13]:
model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))

## 3.4. Çıktı katmanını ekleme

In [14]:
# units: 1 (binary olduğu için)
# activation: sigmoid

In [15]:
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## 3.5. Ağı Derleme (Compiling the model)

In [20]:
# Optimizer: rmsprop (diğer ağlarda genelde adam, ancak RNN'de en çok tavsiye edilen rmsprop)
# loss: binary_crossentropy 
# metrics: accuracy

In [21]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
             metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 128)          2560000   
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(X_train, y_train, epochs=3, batch_size=128)

Epoch 1/3
25000/25000 [==============================] - 24s 970us/sample - loss: 0.4622 - accuracy: 0.7824
Epoch 2/3
25000/25000 [==============================] - 25s 999us/sample - loss: 0.2908 - accuracy: 0.8823
Epoch 3/3
25000/25000 [==============================] - 25s 1ms/sample - loss: 0.2336 - accuracy: 0.9094


## 3.6. Modeli Değerlendirme

In [24]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

25000/25000 [==============================] - 16s 627us/sample - loss: 0.3376 - accuracy: 0.8537


In [25]:
print("test_loss", test_loss)
print("test_accuracy", test_accuracy)

test_loss 0.3375608405780792
test_accuracy 0.85368
